In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import datetime
import h3pandas

/scicore/home/roeoesli/valipo0000/training/anaconda3/envs/pochas_env/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [5]:
import xarray as xr
import s3fs
fs_s3 = s3fs.S3FileSystem(anon=True)
s3path = 'era5-pds/2022/05/data/precipitation_amount_1hour_Accumulation.nc'
remote_file_obj = fs_s3.open(s3path, mode='rb')
ds = xr.open_dataset(remote_file_obj, engine='h5netcdf')

EndpointConnectionError: Could not connect to the endpoint URL: "https://era5-pds.s3.us-east-1a.amazonaws.com/2022/05/data/precipitation_amount_1hour_Accumulation.nc"

In [4]:
ds = xr.open_dataset('precipitation_amount_1hour_Accumulation.nc')

/scicore/home/roeoesli/valipo0000/training/anaconda3/envs/pochas_env/lib/python3.8/site-packages/xarray/backends/plugins.py:65: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [ ]:
ds

In [ ]:
ds.f'{list(ds.keys())[1]}'

In [ ]:
list(ds.coords)

In [5]:
latitudes = ds.lon.values
longitudes = ds.lat.values
times = ds.time1.values
precipitation_amount_1hour_Accumulation = ds.precipitation_amount_1hour_Accumulation.values

In [6]:
times_grid, latitudes_grid, longitudes_grid = [x.flatten() for x in np.meshgrid(times, latitudes, longitudes, indexing='ij')]

In [7]:
times_grid.shape, latitudes_grid.shape, latitudes_grid.shape, precipitation_amount_1hour_Accumulation.shape

((772450560,), (772450560,), (772450560,), (744, 721, 1440))

In [ ]:
times_grid[10000000]

In [8]:
df = pd.DataFrame({
     'time': times_grid,
     'latitude': latitudes_grid,
     'longitude': longitudes_grid,
     'precipitation_amount_1hour_Accumulation': precipitation_amount_1hour_Accumulation.flatten()})

In [9]:
df.tail()

,time,latitude,longitude,precipitation_amount_1hour_Accumulation
772450555,2022-05-31 23:00:00,359.75,-89.00,0.0
772450556,2022-05-31 23:00:00,359.75,-89.25,0.0
772450557,2022-05-31 23:00:00,359.75,-89.50,0.0
772450558,2022-05-31 23:00:00,359.75,-89.75,0.0
772450559,2022-05-31 23:00:00,359.75,-90.00,0.0


In [ ]:
times_grid

In [ ]:
ds.loc['2022-05-01T00:00:00.000000000':'2022-05-01T09:00:00.000000000']

In [ ]:
ds.sel(time1=slice("2022-05-01T00:00:00", "2022-05-01T09:00:00"))

In [ ]:
list_coords = list(ds.coords)

In [ ]:
ds.sel(time_var=slice("2022-05-01T00:00:00", "2022-05-01T09:00:00"))

In [10]:
time_var = {list_coords[2]:slice("2022-05-01T00:00:00", "2022-05-01T09:00:00")}

NameError: name 'list_coords' is not defined

In [ ]:
ds.sel(time_var)

In [ ]:
df.shape

In [ ]:
dfh3 = df.h3.geo_to_h3(resolution=1,lat_col="latitude",lng_col="longitude")

In [11]:
df_ = df.iloc[:10,:].h3.geo_to_h3(resolution=10,lat_col="latitude",lng_col="longitude")

In [ ]:
df_.to_parquet('./test.parquet')

In [ ]:
a = pd.read_parquet('./test.parquet')

In [ ]:
dfh3 = a.drop(columns=['time', 'latitude', 'longitude']).groupby('h3_10').sum()

In [12]:
type(df_)

pandas.core.frame.DataFrame

In [13]:
df_

,time,latitude,longitude,precipitation_amount_1hour_Accumulation
h3_10,,,,
8a86b6236307fff,2022-05-01,0.0,90.00,0.000061
8a86b625c0c7fff,2022-05-01,0.0,89.75,0.000061
8a86b0423447fff,2022-05-01,0.0,89.50,0.000061
8a86b044948ffff,2022-05-01,0.0,89.25,0.000061
8a86b0629877fff,2022-05-01,0.0,89.00,0.000061
8a86b3902887fff,2022-05-01,0.0,88.75,0.000061
8a86b3860c0ffff,2022-05-01,0.0,88.50,0.000061
8a86b3b0dc9ffff,2022-05-01,0.0,88.25,0.000061
8a86b16932e7fff,2022-05-01,0.0,88.00,0.000061


In [ ]:
a.shape

In [ ]:
a

In [ ]:
dfh3